In [5]:
%load_ext autoreload
%autoreload
import mwparserfromhell

#import inspect, os
#currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
#parentdir = os.path.dirname(currentdir)
#os.sys.path.insert(0,parentdir)

#from api.db_util import *
#from rec_util import TopicModel, LDATranslationRecommender, SearchTranslationRecommender, google_search, wiki_search

In [2]:

import mwparserfromhell
import requests

s='en'
title="Biology"
url_template = 'https://%s.wikipedia.org/w/index.php?action=raw&title=%s'
url = url_template % (s, title)
text = requests.get(url).text
wikicode = mwparserfromhell.parse(text)

In [3]:
nodes = []
for t in wikicode.filter_text():
    nodes.append(t)
    

In [4]:
for t in wikicode.filter_templates():
    wikicode.remove(t)
for t in wikicode.filter_comments():
    wikicode.remove(t)
for t in wikicode.filter_external_links():
    wikicode.remove(t)
for t in wikicode.filter_headings():
    wikicode.remove(t)
for t in wikicode.filter_wikilinks():
    wikicode.remove(t)
for t in wikicode.filter_tags():
    wikicode.remove(t)


ValueError: [[Robert Hooke|Robert Hooke's]]

In [56]:
wikicode.nodes

['\n__NOTOC__\nThis collection of ',
 ' collects the names of topics related to ',
 '.  Everything related to law, even quite remotely, should be included on the alphabetical list, and on the appropriate topic lists.  All links on topical lists should also appear in the main alphabetical listing.  The process of creating lists is ongoing',
 '&nbsp;',
 '– these lists are neither complete nor up-to-date',
 '&nbsp;',
 "– if you see an article that should be listed but is not (or one that shouldn't be listed as legal but is), please update the lists accordingly.\n\n",
 '\n',
 '&nbsp;',
 '–\n',
 '\n\n',
 '\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp;',
 '–\n',
 '&nbsp

In [36]:
type(parsed_text)

mwparserfromhell.wikicode.Wikicode

In [2]:
def google_search(s, article, n = 10):
    q = 'site:%s.wikipedia.org %s' % (s, article)
    results = list(search(q, stop=n))
    main_articles = []
    for r in results:
        if r.startswith('https://en.wikipedia.org/wiki'):
            r = r[30:]
            if ':' not in r and '%3' not in r:
                main_articles.append(r)
    return main_articles
                
def wiki_search(s, article, n = 10):
    mw_api = 'https://%s.wikipedia.org/w/api.php' % s
    params = {
        'action': 'query',
        'list': 'search',
        'format': 'json',
        'srsearch': article,
        'srnamespace' : 0,
        'srwhat': 'text',
        'srprop': 'wordcount',
        'srlimit': n

    }
    response = requests.get(mw_api, params=params).json()['query']['search']
    return [r['title'].replace(' ', '_') for r in response]  

In [35]:
titles = wiki_search(s, 'disambiguation', n = 10)

In [36]:
titles

['Word-sense_disambiguation',
 'Persians',
 'Avast',
 'Sealand',
 'Civil',
 'Router',
 'Connor',
 'Turkic',
 'H_index',
 'Chuuk']

In [37]:
def filter_disambiguation(s, titles):
    query = 'https://%s.wikipedia.org/w/api.php?action=query&prop=pageprops&ppprop=disambiguation&format=json&titles=%s' % (s, '|'.join(titles))
    response = requests.get(query).json()
    disambiguation = set()
    for k,v in response['query']['pages'].items():
        if 'pageprops' in v and 'disambiguation' in v['pageprops']:
            title = v['title'].replace(' ', '_')
            disambiguation.add(title)
    return disambiguation

In [38]:
filter_disambiguation(s, titles)
    

{'Avast',
 'Chuuk',
 'Civil',
 'Connor',
 'H_index',
 'Persians',
 'Router',
 'Sealand',
 'Turkic'}

In [20]:
response['query']['pages']

{'17484233': {'ns': 0, 'pageid': 17484233, 'title': 'Table apple'},
 '1810911': {'ns': 0, 'pageid': 1810911, 'title': 'Cortland (apple)'},
 '18978754': {'ns': 0, 'pageid': 18978754, 'title': 'Apple'},
 '25188291': {'ns': 0, 'pageid': 25188291, 'title': 'Eva (apple)'},
 '36071326': {'ns': 0,
  'pageid': 36071326,
  'pageprops': {'disambiguation': ''},
  'title': 'Apple (disambiguation)'},
 '42148574': {'ns': 0, 'pageid': 42148574, 'title': 'Creston (apple)'},
 '44906236': {'ns': 0,
  'pageid': 44906236,
  'title': 'Liveland Raspberry apple'},
 '7123202': {'ns': 0, 'pageid': 7123202, 'title': 'Paula Red'},
 '856': {'ns': 0, 'pageid': 856, 'title': 'Apple Inc.'},
 '8825454': {'ns': 0, 'pageid': 8825454, 'title': 'Mutsu (apple)'}}

In [8]:
query = 'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=%swiki&titles=%s&props=statements/urls&format=json' % (s, '|'.join(titles))
response = requests.get(query).json()

In [9]:
response

{'entities': {'Q158657': {'id': 'Q158657', 'type': 'item'},
  'Q16969204': {'id': 'Q16969204', 'type': 'item'},
  'Q19597760': {'id': 'Q19597760', 'type': 'item'},
  'Q227418': {'id': 'Q227418', 'type': 'item'},
  'Q2613423': {'id': 'Q2613423', 'type': 'item'},
  'Q2998137': {'id': 'Q2998137', 'type': 'item'},
  'Q312': {'id': 'Q312', 'type': 'item'},
  'Q3395974': {'id': 'Q3395974', 'type': 'item'},
  'Q5414989': {'id': 'Q5414989', 'type': 'item'},
  'Q7154755': {'id': 'Q7154755', 'type': 'item'}},
 'success': 1,
 'warnings': {'wbgetentities': {'*': "Unrecognized value for parameter 'props': statements/urls"}}}

In [4]:
import requests
s='en'
t = 'ca'

def filter_missing(s, t, titles):
    query = 'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=%swiki&titles=%s&props=sitelinks/urls&format=json' % (s, '|'.join(titles))
    response = requests.get(query).json()
    missing = {}
    swiki = '%swiki' % s
    twiki = '%swiki' % t
    for k, v in response['entities'].items():
        if 'sitelinks' in v:
            if swiki in v['sitelinks'] and twiki not in v['sitelinks']:
                title = v['sitelinks'][swiki]['title'].replace(' ', '_')
                print(title)
                missing[title] = k
    return missing

In [5]:
from datetime import datetime
from dateutil import relativedelta

    
def get_article_views(s, article):
    start = (datetime.utcnow() - relativedelta.relativedelta(days=1)).strftime('%Y%m%d00')
    stop = (datetime.utcnow() - relativedelta.relativedelta(days=15)).strftime('%Y%m%d00')
    query = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/%s.wikipedia/all-access/user/%s/daily/%s/%s"
    query = query % (s, article, stop, start)
    response = requests.get(query).json()
    return sum([x['views'] for x in response['items']])




In [167]:


def get_top_article_views(s):
    dt = (datetime.utcnow() - relativedelta.relativedelta(days=2)).strftime('%Y/%m/%d')
    query = "https://wikimedia.org/api/rest_v1/metrics/pageviews/top/%s.wikipedia/all-access/%s" % (s, dt)
    response = requests.get(query).json()
    article_pv_dict = OrderedDict()
    for d in json.loads(response['items'][0]['articles']):
        if 'article' in d and ':' not in d['article'] and not d['article'].startswith('List'):
            article_pv_dict[d['article']] =  d['views']
    return article_pv_dict

In [31]:
import multiprocessing as mp

def get_article_views(arg_tuple):
        s = arg_tuple[0]
        article = arg_tuple[1]
        start = (datetime.utcnow() - relativedelta.relativedelta(days=1)).strftime('%Y%m%d00')
        stop = (datetime.utcnow() - relativedelta.relativedelta(days=15)).strftime('%Y%m%d00')
        query = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/%s.wikipedia/all-access/user/%s/daily/%s/%s"
        query = query % (s, article, stop, start)
        response = requests.get(query).json()
        return (article, sum([x['views'] for x in response['items']]))


def get_article_views_parallel(s, articles):
    p = mp.Pool(len(articles))
    return dict(p.map(get_article_views, [(s, a) for a in articles]))

In [39]:
def get_article_views_threaded(s, articles):
    with concurrent.futures.ThreadPoolExecutor(8) as executor:
        result = executor.map(get_article_views, [(s, a) for a in articles])
        return dict(result)

In [40]:
get_article_views_parallel('en', titles)

{'APPLE': 459,
 'Apple': 75904,
 'Apple_Inc.': 218134,
 'Apple_crisp': 9642,
 'Cortland_(apple)': 2221,
 'Creston_(apple)': 77,
 'Eva_(apple)': 91,
 'Liveland_Raspberry_apple': 127,
 'Mutsu_(apple)': 3221,
 'Table_apple': 500}

In [46]:
g = get_article_views_threded(s, titles)

In [47]:
dict(g)

{'APPLE': 459,
 'Apple': 75904,
 'Apple_Inc.': 218134,
 'Apple_crisp': 9642,
 'Cortland_(apple)': 2221,
 'Creston_(apple)': 77,
 'Eva_(apple)': 91,
 'Liveland_Raspberry_apple': 127,
 'Mutsu_(apple)': 3221,
 'Table_apple': 500}

In [65]:
filter_missing(s, t, titles)

['Acemannan',
 'Forever_Living_Products',
 'Aloe_emodin',
 'International_Aloe_Science_Council',
 'Sunburn',
 'Rex_Maughan']

In [36]:
titles

['Aloe_vera',
 'Acemannan',
 'Forever_Living_Products',
 'Aloe',
 'Sap',
 'Aloe_emodin',
 'International_Aloe_Science_Council',
 'Aloe_arborescens',
 'Sunburn',
 'Rex_Maughan']

Sunburn
Aloe emodin
International Aloe Science Council
Acemannan
Rex Maughan


In [8]:
article = 'aloe vera'
titles = wiki_search('en', article)
print(titles)

['Aloe_vera', 'Acemannan', 'Forever_Living_Products', 'Aloe', 'Sap', 'Aloe_emodin', 'International_Aloe_Science_Council', 'Aloe_arborescens', 'Sunburn', 'Rex_Maughan']


In [6]:
try:
    print (google_search('en', article))
except:
    print('upps')

['Aloe_vera', 'Succulent_plant', 'Medicinal_plants', 'Xanthorrhoeaceae', 'Aloe', 'Aloe_buettneri', 'List_of_Aloe_species', 'Aloe_arborescens', 'Aloin', 'Acemannan', 'Aloe_broomii']


/Users/ellerywulczyn/miniconda3/lib/python3.4/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


# Search Recommender

In [37]:
data_dir = '/Users/ellerywulczyn/translation-recs-app/data' # directory were the topic model data lives
s = 'en' # source wiki
t = 'it' # target wiki

In [38]:
tr = SearchTranslationRecommender(data_dir, s, t) # s->t missing article recommender

In [40]:
'Albert_Einstein' in tr.missing_df.index

False

In [44]:
tr.get_seeded_recommendations('albert einstein', num_recs=100)

[{'pageviews': 27321,
  'title': "Albert_Einstein's_brain",
  'wikidata_id': 'Q2464312'},
 {'pageviews': 23986, 'title': 'Einstein_family', 'wikidata_id': 'Q3292175'},
 {'pageviews': 20103, 'title': 'Eduard_Einstein', 'wikidata_id': 'Q118253'},
 {'pageviews': 13444,
  'title': 'Bernhard_Caesar_Einstein',
  'wikidata_id': 'Q824855'},
 {'pageviews': 7768,
  'title': 'Albert_Einstein_College_of_Medicine',
  'wikidata_id': 'Q2030894'},
 {'pageviews': 5131, 'title': 'Evelyn_Einstein', 'wikidata_id': 'Q432375'},
 {'pageviews': 4976,
  'title': 'Political_views_of_Albert_Einstein',
  'wikidata_id': 'Q4710118'},
 {'pageviews': 4446,
  'title': 'Albert_Einstein_in_popular_culture',
  'wikidata_id': 'Q4710136'},
 {'pageviews': 4269, 'title': 'Why_Socialism?', 'wikidata_id': 'Q3400616'},
 {'pageviews': 3612,
  'title': 'Albert_Einstein_Award',
  'wikidata_id': 'Q577432'},
 {'pageviews': 2547,
  'title': 'Albert_Einstein_House',
  'wikidata_id': 'Q4710127'},
 {'pageviews': 2265,
  'title': "Einste

# Load Models

In [5]:
data_dir = '/Users/ellerywulczyn/translation-recs-app/data' # directory were the topic model data lives
s = 'en' # source wiki
t = 'it' # target wiki

In [4]:
tm = TopicModel(data_dir, s) # source language topic model

In [6]:
tr = LDATranslationRecommender(data_dir, s, t, tm) # s->t missing article recommender

NameError: name 'tm' is not defined

# Get Personalized Recs

In [39]:
contribution_df = get_user_contributions('enwiki', 'DarTar', window = 15)

In [40]:
contribution_df

,bytes_added,page_id,page_title,timestamp,wikidata_id
0,1317,8327348,Earthquake_warning_system,20150825052755,Q5327282
1,16,18197718,Earthquake_Early_Warning_(Japan),20150825052429,Q1281628
2,85,60340,Bay_Area_Rapid_Transit,20150824023726,Q610120
3,3928,43645203,2014_South_Napa_earthquake,20150823171140,Q17602018
4,1888,43668436,Berkeley_Seismological_Laboratory,20150823161130,Q17985847
5,48,13961020,List_of_California_wildfires,20150802200740,Q6565566
6,2124,233983,Military_robot,20150728023916,Q1235833
7,770,5570834,Piazza_della_Loggia_bombing,20150723024515,Q3975758
8,223,1102205,Cascadia_subduction_zone,20150722033415,Q4024718
9,6,29919086,Aloe_haemanthifolia,20150715161459,Q143622


In [42]:
tr.get_user_recommendations(contribution_df, num_recs=10, min_score=0.0)

[{'pageviews': 10757,
  'title': 'Tsunami_warning_system',
  'wikidata_id': 'Q2744117'},
 {'pageviews': 6889,
  'title': 'Earthquake_warning_system',
  'wikidata_id': 'Q5327282'},
 {'pageviews': 1965,
  'title': 'Indian_Ocean_Tsunami_Warning_System',
  'wikidata_id': 'Q3509205'},
 {'pageviews': 302, 'title': 'Smith_Dharmasaroja', 'wikidata_id': 'Q7545126'},
 {'pageviews': 143,
  'title': 'Berkeley_Seismological_Laboratory',
  'wikidata_id': 'Q17985847'},
 {'pageviews': 196,
  'title': 'California_Earthquake_Prediction_Evaluation_Council',
  'wikidata_id': 'Q5020464'},
 {'pageviews': 542,
  'title': 'National_Tsunami_Warning_Center',
  'wikidata_id': 'Q7984847'},
 {'pageviews': 1974,
  'title': 'Timeline_of_the_2004_Indian_Ocean_earthquake',
  'wikidata_id': 'Q7806246'},
 {'pageviews': 424,
  'title': 'California_landslides',
  'wikidata_id': 'Q5021271'},
 {'pageviews': 200,
  'title': 'CIA_transnational_health_and_economic_activities',
  'wikidata_id': 'Q5011559'}]

# Most Viewed Missing Articles

In [43]:
tr.get_global_recommendations(num_recs = 10)

[{'pageviews': 3795620,
  'title': 'Directoire_style',
  'wikidata_id': 'Q1227514'},
 {'pageviews': 3287042,
  'title': 'Bajrangi_Bhaijaan',
  'wikidata_id': 'Q18603021'},
 {'pageviews': 2558218,
  'title': 'Suicide_Squad_(film)',
  'wikidata_id': 'Q18604504'},
 {'pageviews': 2087343, 'title': '2015_in_film', 'wikidata_id': 'Q13553713'},
 {'pageviews': 1667859,
  'title': 'Battleground_(2015)',
  'wikidata_id': 'Q18890191'},
 {'pageviews': 1653919,
  'title': '2014–15_English_Premiership_(rugby_union)',
  'wikidata_id': 'Q17101036'},
 {'pageviews': 1420741,
  'title': 'Bobbi_Kristina_Brown',
  'wikidata_id': 'Q18951229'},
 {'pageviews': 1404357, 'title': 'Dan_Bilzerian', 'wikidata_id': 'Q16199296'},
 {'pageviews': 1351601, 'title': 'Fetty_Wap', 'wikidata_id': 'Q19361511'},
 {'pageviews': 1321719, 'title': 'Ramadan', 'wikidata_id': 'Q41662'}]

# Get Seeded Recommendations

In [44]:
tr.get_seeded_recommendations( 'Ebola_virus_disease', num_recs=10, min_score=0.1)

[{'pageviews': 2167, 'title': 'Bosutinib', 'wikidata_id': 'Q894611'},
 {'pageviews': 272, 'title': 'Bovine_coronavirus', 'wikidata_id': 'Q4950554'},
 {'pageviews': 195, 'title': 'Sacrosidase', 'wikidata_id': 'Q7397389'},
 {'pageviews': 6109, 'title': 'Insulin_lispro', 'wikidata_id': 'Q3492616'},
 {'pageviews': 470, 'title': 'Alicaforsen', 'wikidata_id': 'Q17147922'},
 {'pageviews': 19576,
  'title': 'Hypervitaminosis_A',
  'wikidata_id': 'Q3100515'},
 {'pageviews': 586, 'title': 'Beractant', 'wikidata_id': 'Q4890968'},
 {'pageviews': 308, 'title': 'Cicletanine', 'wikidata_id': 'Q5119443'},
 {'pageviews': 1411, 'title': 'Streptococcus_suis', 'wikidata_id': 'Q287694'},
 {'pageviews': 9609, 'title': 'Benfotiamine', 'wikidata_id': 'Q409953'}]

In [45]:
tr.get_seeded_recommendations( 'Biology', num_recs=10, min_score=0.1)

[{'pageviews': 209, 'title': 'Halocin', 'wikidata_id': 'Q5643356'},
 {'pageviews': 1612, 'title': 'Phenome', 'wikidata_id': 'Q844116'},
 {'pageviews': 1240, 'title': 'Weismann_barrier', 'wikidata_id': 'Q1883991'},
 {'pageviews': 1964,
  'title': 'Common_misunderstandings_of_genetics',
  'wikidata_id': 'Q5153572'},
 {'pageviews': 351, 'title': 'Chemoton', 'wikidata_id': 'Q1106696'},
 {'pageviews': 655,
  'title': 'History_of_model_organisms',
  'wikidata_id': 'Q5868220'},
 {'pageviews': 556, 'title': '2R_hypothesis', 'wikidata_id': 'Q4633212'},
 {'pageviews': 844, 'title': 'Two-empire_system', 'wikidata_id': 'Q3324187'},
 {'pageviews': 10017,
  'title': 'Genotype-phenotype_distinction',
  'wikidata_id': 'Q5533517'},
 {'pageviews': 269, 'title': 'Jan_Klein', 'wikidata_id': 'Q822241'}]

In [46]:
tr.get_seeded_recommendations( 'Pad_Thai', num_recs=10, min_score=0.1)

[{'pageviews': 58024, 'title': 'Pad_Thai', 'wikidata_id': 'Q730298'},
 {'pageviews': 4071, 'title': 'Nam_phrik', 'wikidata_id': 'Q4922876'},
 {'pageviews': 918, 'title': 'Nam_khao', 'wikidata_id': 'Q6961172'},
 {'pageviews': 2158, 'title': 'Nam_tok_(food)', 'wikidata_id': 'Q4923638'},
 {'pageviews': 522, 'title': 'Bánh_pía', 'wikidata_id': 'Q5004807'},
 {'pageviews': 5015, 'title': 'Thai_fried_rice', 'wikidata_id': 'Q4922489'},
 {'pageviews': 8795, 'title': 'Lao_cuisine', 'wikidata_id': 'Q1372774'},
 {'pageviews': 10070, 'title': 'Larb', 'wikidata_id': 'Q3052288'},
 {'pageviews': 1057, 'title': 'Nam_ngiao', 'wikidata_id': 'Q130194'},
 {'pageviews': 557, 'title': 'Ohn_no_khao_swè', 'wikidata_id': 'Q7081117'}]

In [47]:
tr.get_seeded_recommendations( 'Red_Skelton', num_recs=10, min_score=0.1)

[{'pageviews': 9764, 'title': 'Mama_(TV_series)', 'wikidata_id': 'Q17099706'},
 {'pageviews': 61318, 'title': 'Jessa_Duggar', 'wikidata_id': 'Q19282024'},
 {'pageviews': 872, 'title': 'Jerry_Lester', 'wikidata_id': 'Q19319436'},
 {'pageviews': 10725, 'title': 'Garry_Moore', 'wikidata_id': 'Q4354677'},
 {'pageviews': 598, 'title': 'Rich_Jeffries', 'wikidata_id': 'Q7323309'},
 {'pageviews': 441, 'title': 'John_Aylesworth', 'wikidata_id': 'Q3180955'},
 {'pageviews': 463, 'title': 'Ira_Skutch', 'wikidata_id': 'Q20065034'},
 {'pageviews': 926, 'title': 'Freda_Wright-Sorce', 'wikidata_id': 'Q5496664'},
 {'pageviews': 429, 'title': 'Eddie_Hubbard', 'wikidata_id': 'Q5336137'},
 {'pageviews': 387, 'title': 'Rasha_Drachkovitch', 'wikidata_id': 'Q7294766'}]

In [48]:
tr.get_seeded_recommendations( 'Biology', num_recs=10, min_score=0.1)

[{'pageviews': 209, 'title': 'Halocin', 'wikidata_id': 'Q5643356'},
 {'pageviews': 1612, 'title': 'Phenome', 'wikidata_id': 'Q844116'},
 {'pageviews': 1240, 'title': 'Weismann_barrier', 'wikidata_id': 'Q1883991'},
 {'pageviews': 1964,
  'title': 'Common_misunderstandings_of_genetics',
  'wikidata_id': 'Q5153572'},
 {'pageviews': 351, 'title': 'Chemoton', 'wikidata_id': 'Q1106696'},
 {'pageviews': 655,
  'title': 'History_of_model_organisms',
  'wikidata_id': 'Q5868220'},
 {'pageviews': 556, 'title': '2R_hypothesis', 'wikidata_id': 'Q4633212'},
 {'pageviews': 844, 'title': 'Two-empire_system', 'wikidata_id': 'Q3324187'},
 {'pageviews': 10017,
  'title': 'Genotype-phenotype_distinction',
  'wikidata_id': 'Q5533517'},
 {'pageviews': 269, 'title': 'Jan_Klein', 'wikidata_id': 'Q822241'}]

In [49]:
tr.get_seeded_recommendations( 'Individualism', num_recs=10, min_score=0.1)

[{'pageviews': 2200,
  'title': 'Existentialist_anarchism',
  'wikidata_id': 'Q5420363'},
 {'pageviews': 943, 'title': 'Politics_of_Nature', 'wikidata_id': 'Q3395235'},
 {'pageviews': 293,
  'title': 'The_Right_to_Be_Greedy',
  'wikidata_id': 'Q7760769'},
 {'pageviews': 261,
  'title': 'The_Ecology_of_Freedom',
  'wikidata_id': 'Q18207980'},
 {'pageviews': 2969,
  'title': 'Influences_on_Karl_Marx',
  'wikidata_id': 'Q6030251'},
 {'pageviews': 366,
  'title': 'Philosophy_and_Real_Politics',
  'wikidata_id': 'Q7186224'},
 {'pageviews': 3745, 'title': 'Transcendentals', 'wikidata_id': 'Q1432095'},
 {'pageviews': 249, 'title': 'Epistemocracy', 'wikidata_id': 'Q5383596'},
 {'pageviews': 967,
  'title': 'Liberalism_and_the_Limits_of_Justice',
  'wikidata_id': 'Q6540882'},
 {'pageviews': 4064, 'title': 'Post-left_anarchy', 'wikidata_id': 'Q2574500'}]

In [1]:
from api_based_rec import *

In [12]:
top_article_pv_dict = get_top_article_views('en')

In [19]:
top_article_pv_dict = OrderedDict(sorted(top_article_pv_dict.items(), key=lambda x: random.random()))


In [20]:
top_article_pv_dict

OrderedDict([('Rudyard_Kipling', 13814),
             ('Triple_9', 11882),
             ('Saúl_Álvarez', 21470),
             ('Spire_of_Dublin', 14977),
             ('Chemtrail_conspiracy_theory', 39266),
             ('Kate_Hudson', 10299),
             ('Snoop_Dogg', 11526),
             ('Vanity_(singer)', 50430),
             ('Twenty_One_Pilots', 14664),
             ('1999_(song)', 78817),
             ('Elon_Musk', 17287),
             ('The_Durrells', 14255),
             ('Dirty_Mind', 27850),
             ('Spy_(2015_film)', 10373),
             ('Natalie_Dormer', 24628),
             ('The_Most_Beautiful_Girl_in_the_World_(Prince_song)', 13178),
             ('Mike_Tyson', 12138),
             ('Gloria_Vanderbilt', 20096),
             ('Mark_Zuckerberg', 11336),
             ('Pornhub', 21775),
             ('Love_Symbol_Album', 21619),
             ('Sunny_Leone', 26968),
             ('Shaun_Livingston', 10941),
             ('UFC_199', 19650),
             ('Kim_Basing

In [2]:
get_global_recommendations('en', 'ar', 10)

([{'pageviews': 12264,
   'title': '1990_FA_Cup_Final',
   'wikidata_id': 'Q4484517'},
  {'pageviews': 14177,
   'title': 'The_Big_Bang_Theory_(season_9)',
   'wikidata_id': 'Q20853388'},
  {'pageviews': 28131,
   'title': 'St_Albans_School,_Hertfordshire',
   'wikidata_id': 'Q7592283'},
  {'pageviews': 11573, 'title': 'Janam_TV', 'wikidata_id': 'Q20950321'},
  {'pageviews': 13756,
   'title': 'Musicology_(album)',
   'wikidata_id': 'Q592659'},
  {'pageviews': 11592,
   'title': 'The_Twilight_Saga_(film_series)',
   'wikidata_id': 'Q652599'},
  {'pageviews': 21470, 'title': 'Saúl_Álvarez', 'wikidata_id': 'Q2117769'},
  {'pageviews': 13396, 'title': 'Raspberry_Beret', 'wikidata_id': 'Q1138900'},
  {'pageviews': 12839,
   'title': 'Indian_Premier_League',
   'wikidata_id': 'Q396412'},
  {'pageviews': 11822, 'title': 'Billy_Paul', 'wikidata_id': 'Q863212'}],
 None)

In [10]:
sorted([1,3,1], key = lambda x: random.random())

[3, 1, 1]